In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


groq_api_key = os.getenv("API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B10E3E97D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B10E3F0390>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage,SystemMessage

model.invoke([HumanMessage(content="Hi, My name is Arjun and Iam a Gen AI Developer")])

AIMessage(content="Hi Arjun! It's nice to meet you.\n\nThat's awesome that you're a Gen AI Developer! It's such an exciting field right now. \n\nWhat kind of projects are you working on?  I'm always eager to hear about what people are doing with generative AI. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 22, 'total_tokens': 91, 'completion_time': 0.125454545, 'prompt_time': 0.002135825, 'queue_time': 0.24263195799999998, 'total_time': 0.12759037}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--3162f025-faf6-4a00-86b2-9ab300af7e3b-0', usage_metadata={'input_tokens': 22, 'output_tokens': 69, 'total_tokens': 91})

In [8]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, My name is Arjun and Iam a Gen AI Developer"),
    AIMessage(content="Hi Arjun,\n\nIt's nice to meet you! That's awesome that you're a Gen AI Developer. It's a really exciting field right now.\n\nWhat kind of projects are you working on?  I'm always interested to hear about what people are building with AI.\n"),
    HumanMessage(content="Hey Whats my name and what do i do?")
])

AIMessage(content="You are Arjun, and you are a Gen AI Developer!  \n\nIs there anything else you'd like to know or talk about? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 104, 'total_tokens': 137, 'completion_time': 0.06, 'prompt_time': 0.005529318, 'queue_time': 0.245730632, 'total_time': 0.065529318}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--5aea5b11-ed5d-4470-896b-3fe0a9e2b39b-0', usage_metadata={'input_tokens': 104, 'output_tokens': 33, 'total_tokens': 137})

In [11]:
## Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store ={}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model,get_session_history)


In [10]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Arjun and Iam a Gen AI Developer")],
    config=config
)

In [14]:
response.content

"Hi Arjun,\n\nIt's great to meet you! Being a Gen AI developer is fascinating. What kind of projects are you working on these days? I'm always eager to learn about the innovative things people are building with this technology.  \n\nPerhaps you could tell me about:\n\n* **The specific type of Gen AI you focus on:** (e.g., text generation, image creation, code generation)\n* **A project you're particularly proud of:**\n* **Any challenges you're facing in your work:**\n\nI'm here to listen and learn!  😊  \n\n"

In [15]:
with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config
)

AIMessage(content='Your name is Arjun, as you told me at the beginning of our conversation.  😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 234, 'total_tokens': 256, 'completion_time': 0.04, 'prompt_time': 0.00913723, 'queue_time': 0.243186939, 'total_time': 0.04913723}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--3a6d65c1-09b8-4b6c-8c8e-37e4799cab96-0', usage_metadata={'input_tokens': 234, 'output_tokens': 22, 'total_tokens': 256})

In [17]:
config1={"configurable":{"session_id":"chat2"}}
response =with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)

response.content


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [19]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant, Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model